In [232]:
from lib.actor import Actor
from lib.input_model import *
import pandas as pd
from collections import Counter, defaultdict
from scipy.stats import mode

Example 4.1 Grassroot democracy

In [2]:
group_dict = {0:["1","2","3","4","5","6","7"]}
topic_dict = {0:3,1:3,2:3}

In [3]:
generate_motivation_params = {"method":"manual", "motivations":{"1":0.8, "2":0.52, "3":0.3, "4":0.44, 
                                                                  "5":0.42, "6":0.57, "7":0.36}}

In [4]:
beliefs = {
    "1": {(1,1):1.35, (1,2):2.47, (1,3):2.54,
          (2,1):0.19, (2,2):2.49, (2,3):1.75,
          (3,1):1.95, (3,2):1.10, (3,3):2.67},
    "2": {(1,1):0.88, (1,2):2.03, (1,3):0.67,
          (2,1):0.90, (2,2):0.64, (2,3):1.04,
          (3,1):1.46, (3,2):1.71, (3,3):2.20},
    "3": {(1,1):0.20, (1,2):2.59, (1,3):1.50,
          (2,1):0.11, (2,2):2.08, (2,3):2.51,
          (3,1):1.75, (3,2):1.60, (3,3):0.16},
    "4": {(1,1):1.22, (1,2):2.36, (1,3):1.87,
          (2,1):2.72, (2,2):2.22, (2,3):2.25,
          (3,1):1.49, (3,2):2.70, (3,3):0.03},
    "5": {(1,1):0.67, (1,2):0.40, (1,3):0.04,
          (2,1):2.52, (2,2):1.59, (2,3):1.81,
          (3,1):2.17, (3,2):0.97, (3,3):0.75},
    "6": {(1,1):0.57, (1,2):0.66, (1,3):1.98,
          (2,1):2.06, (2,2):1.35, (2,3):1.42,
          (3,1):1.79, (3,2):0.26, (3,3):0.87},
    "7": {(1,1):2.67, (1,2):2.03, (1,3):1.81,
          (2,1):0.43, (2,2):0.05, (2,3):2.38,
          (3,1):2.39, (3,2):2.08, (3,3):1.29},
}

In [5]:
generate_belief_params = {"method":"manual", "beliefs":beliefs}

In [6]:
topic_relevances = {
    "1": {1:0.95, 2:0.78, 3:0.68},
    "2": {1:0.86, 2:0.44, 3:0.28},
    "3": {1:0.52, 2:0.02, 3:0.37},
    "4": {1:0.20, 2:0.93, 3:0.20},
    "5": {1:0.52, 2:0.21, 3:0.52},
    "6": {1:0.50, 2:0.46, 3:0.74},
    "7": {1:0.37, 2:0.41, 3:0.31},
}

In [7]:
generate_topic_relevance_params = {"method":"manual", "topic_relevances": topic_relevances}

In [8]:
config={}
config["generate_motivation_params"] = generate_motivation_params
config["generate_belief_params"] = generate_belief_params
config["generate_topic_relevance_params"] = generate_topic_relevance_params

In [9]:
actors = []
numOfActors = 7

In [10]:
def _allocate_actors():
    motivations = generate_motivation(numOfActors,**config["generate_motivation_params"])
    beliefs = generate_belief(numOfActors,**config["generate_belief_params"])
    topic_relevances = generate_topic_relevance(numOfActors, **config["generate_topic_relevance_params"])
        
    for groupId,actor_id_list in group_dict.items():
        for actor_id in actor_id_list:
            motivation = motivations[actor_id]
            belief = beliefs[actor_id]
            topic_relevance = topic_relevances[actor_id]
            actors.append(Actor(groupId=groupId,motivation=motivation,
                                belief=belief,_id=actor_id,topic_relevance=topic_relevance))

In [11]:
_allocate_actors()

In [12]:
actors

[<lib.actor.Actor instance at 0x7fba9169df38>,
 <lib.actor.Actor instance at 0x7fba9169df80>,
 <lib.actor.Actor instance at 0x7fba9169dfc8>,
 <lib.actor.Actor instance at 0x7fba9169b050>,
 <lib.actor.Actor instance at 0x7fba9169b098>,
 <lib.actor.Actor instance at 0x7fba9169b0e0>,
 <lib.actor.Actor instance at 0x7fba9169b128>]

In [15]:
actors[0].policy_rank

value  rank
topic_id policy_id              
1        1          1.2825     3
         2          2.3465     2
         3          2.4130     1
2        1          0.1482     3
         2          1.9422     1
         3          1.3650     2
3        1          1.3260     2
         2          0.7480     3
         3          1.8156     1

### Ballot Engine

In [281]:
def my_mode_func(x):
    new_column = x.ix[:,-1]
    res = new_column.mode()
    if res.empty:
        # pd mode Empty if nothing has 2+ occurrences
        #scipy.stats.mode
        res = mode(new_column)[0]
        return res
    else:
        return res.values


class BallotEngine:

    def ballot(actors):
        for topic_id in topic_ids:
            # initialize ballot_result for this topic_id
            # multiple rounds possible if a tie
            select_round = 1
            ballot_result = pd.DataFrame({"actor_id":[actor._id for actor in actors]}).set_index("actor_id")
            # call ballot_one_topic recursively
            selected_policy_id, ballot_result = BallotEngine.ballot_one_topic(topic_id,
                                                                             actors,
                                                                             select_round,
                                                                             ballot_result)
            
    @staticmethod
    def ballot_one_topic(topic_id, actors, select_round, ballot_result):
        # collect votes to topic_id
        # first round, rank=1
        
        
        ballot_votes = {actor._id: int(pd.pivot_table(actor.policy_rank.reset_index(), index=["topic_id", "rank"], values=["policy_id"]).ix[(topic_id,select_round)]) for actor in actors}
        print ballot_votes
        
        ballot_result = pd.concat([ballot_result, pd.DataFrame({"actor_id": ballot_votes.keys(), "round %s" %(select_round,):ballot_votes.values()}).set_index("actor_id")], axis=1)
        if select_round==2:
            print ("hi")

        selected_policy_id_list = list(my_mode_func(ballot_result))
        
        print (len(selected_policy_id_list))
        print (selected_policy_id_list)
        # check if run-off
        if len(selected_policy_id_list)>1:
            # remain set of actor_id
            remain_actors_id = set(ballot_result[~ballot_result["round %s" %(select_round,)].isin(selected_policy_id_list)].index.values)
            select_round += 1
            remain_actors = [actor for actor in actors if actor._id in remain_actors_id]
            
            return BallotEngine.ballot_one_topic(topic_id, remain_actors, select_round, ballot_result)
            
        else:
            return selected_policy_id_list, ballot_result

In [280]:
print (actors)
select_round = 1
print (select_round)
ballot_result = pd.DataFrame({"actor_id":[actor._id for actor in actors]}).set_index("actor_id")
print (ballot_result)

[<lib.actor.Actor instance at 0x7fba9169df38>, <lib.actor.Actor instance at 0x7fba9169df80>, <lib.actor.Actor instance at 0x7fba9169dfc8>, <lib.actor.Actor instance at 0x7fba9169b050>, <lib.actor.Actor instance at 0x7fba9169b098>, <lib.actor.Actor instance at 0x7fba9169b0e0>, <lib.actor.Actor instance at 0x7fba9169b128>]
1
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 6, 7]


In [282]:
x = BallotEngine.ballot_one_topic(2,actors,select_round,ballot_result)

{'1': 2, '3': 3, '2': 3, '5': 1, '4': 1, '7': 3, '6': 1}
2
[1, 3]
{'1': 3}
hi
1
[3.0]


In [285]:
x[1]

,round 1,round 2
1,2,3
2,3,NaN
3,3,NaN
4,1,NaN
5,1,NaN
6,1,NaN
7,3,NaN


In [268]:
def my_mode_func(x):
    new_column = x.ix[:,-1]
    res = new_column.mode()
    if res.empty:
        # pd mode Empty if nothing has 2+ occurrences
        #scipy.stats.mode
        res = mode(new_column)[0]
        return res
    else:
        return res.values

In [276]:
pd.DataFrame(x.ix[:,0])

,round 1
1,2
2,3
3,3
4,1
5,1
6,1
7,3


In [278]:
print (type(my_mode_func(pd.DataFrame(x.ix[:,0]))))

<type 'numpy.ndarray'>


In [193]:
y.ix[:,-1].mode().values

array([1, 3])

In [249]:
mode([2,3,3,1,1,1,3])

ModeResult(mode=array([1]), count=array([3]))

In [149]:
initial_ballot_result = pd.DataFrame({"actor_id":[actor._id for actor in actors]}).set_index("actor_id")

In [150]:
initial_ballot_result

""
actor_id
1
2
3
4
5
6
7


#### ballot

In [141]:
ballot_votes = {actor._id:pd.pivot_table(actor.policy_rank.reset_index(),index=["topic_id", "rank"],values=["policy_id"]).ix[(2,1)].values[0] for actor in actors}

In [142]:
ballot_votes

{'1': 2, '2': 3, '3': 3, '4': 1, '5': 1, '6': 1, '7': 3}

In [131]:
ballot_result = pd.DataFrame({"actor_id": ballot_votes.keys(), "selected_policy_id":ballot_votes.values()}).set_index("actor_id")

In [152]:
pd.concat([initial_ballot_result, ballot_result], axis=1)

,selected_policy_id
1,2
2,3
3,3
4,1
5,1
6,1
7,3


In [148]:
ballot_result

,selected_policy_id
actor_id,
1,2
3,3
2,3
5,1
4,1
7,3
6,1


In [161]:
list(ballot_result.mode().values.flatten())

[1, 3]

In [168]:
set(ballot_result[~ballot_result["selected_policy_id"].isin(list(ballot_result.mode().values.flatten()))].index.values)

array(['1'], dtype=object)

In [163]:
ballot_result

,selected_policy_id
actor_id,
1,2
3,3
2,3
5,1
4,1
7,3
6,1
